# Cyclistic Bike-Share Analysis - MySQL Integration

## Loading Data into MySQL Database

**My Objective:** Create a MySQL database and load my cleaned data so I can perform SQL-based analysis and demonstrate database skills.

**My Steps:**
1. Connect to MySQL
2. Create database and table
3. Load cleaned data
4. Verify the data loaded correctly
5. Run analytical SQL queries

---

In [2]:
# Import necessary libraries
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import warnings

# Turn off warning messages
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


---

## Setting Up MySQL Connection

I'll first test my connection to MySQL server to ensure everything is working before proceeding.

**Note:** Make sure MySQL is installed and running on your computer before running this cell.

In [3]:
# Set up MySQL connection parameters
# Update these values to match your MySQL setup
mysql_config = {
    'host': 'localhost',
    'user': 'root',  
    'password': 'mysql',  # Change this to your MySQL password
    'port': 3306
}

# Test the connection to MySQL
try:
    # Attempt to connect
    connection = mysql.connector.connect(**mysql_config)
    
    # Check if connection was successful
    if connection.is_connected():
        print("Successfully connected to MySQL")
        
        # Get and display MySQL version
        version = connection.get_server_info()
        print(f"MySQL Server version: {version}")
        
        # Close the connection
        connection.close()
        
except Exception as e:
    print(f"Error connecting to MySQL: {e}")
    print("\nPlease check:")
    print("1. MySQL is installed and running")
    print("2. Username and password are correct")
    print("3. MySQL server is accessible on the specified port")

Successfully connected to MySQL
MySQL Server version: 8.0.44


---

## Step 1: Create Database

I'll create a new database called `cyclistic_db` to store my bike-share data. If the database already exists, I'll drop it and create a fresh one.

In [4]:
# Create the database
try:
    # Connect to MySQL server
    connection = mysql.connector.connect(**mysql_config)
    cursor = connection.cursor()
    
    # Drop the database if it already exists (to start fresh)
    cursor.execute("DROP DATABASE IF EXISTS cyclistic_db")
    
    # Create a new database
    cursor.execute("CREATE DATABASE cyclistic_db")
    print("Database 'cyclistic_db' created successfully")
    
    # Close cursor and connection
    cursor.close()
    connection.close()
    
except Exception as e:
    print(f"Error creating database: {e}")

Database 'cyclistic_db' created successfully


---

## Step 2: Create Table Schema

Now I'll create a table called `bike_trips` with appropriate data types for each column. This table will hold all my cleaned bike-share data.

In [5]:
# Update connection config to use the new database
mysql_config['database'] = 'cyclistic_db'

# Define the SQL query to create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS bike_trips (
    ride_id VARCHAR(50) PRIMARY KEY,
    rideable_type VARCHAR(50),
    started_at DATETIME,
    ended_at DATETIME,
    start_station_name VARCHAR(255),
    start_station_id VARCHAR(50),
    end_station_name VARCHAR(255),
    end_station_id VARCHAR(50),
    start_lat DECIMAL(10, 8),
    start_lng DECIMAL(11, 8),
    end_lat DECIMAL(10, 8),
    end_lng DECIMAL(11, 8),
    member_casual VARCHAR(20),
    ride_length DECIMAL(10, 2),
    day_of_week INT,
    month INT,
    hour INT
)
"""

# Execute the query to create the table
try:
    # Connect to the database
    connection = mysql.connector.connect(**mysql_config)
    cursor = connection.cursor()
    
    # Create the table
    cursor.execute(create_table_query)
    print("Table 'bike_trips' created successfully")
    
    # Close cursor and connection
    cursor.close()
    connection.close()
    
except Exception as e:
    print(f"Error creating table: {e}")

Table 'bike_trips' created successfully


---

## Step 3: Load Cleaned Data into MySQL

Now I'll load my cleaned dataset into the MySQL database. This is a large dataset (4.8+ million records), so I'll load it in chunks to avoid memory issues.

In [6]:
# Load the cleaned data from the CSV file
df = pd.read_csv('../data/processed/cleaned_2024_data.csv')

# Display how many records we're about to load
total_records = len(df)
print(f"Loading {total_records:,} records into MySQL...")

Loading 4,859,019 records into MySQL...


In [7]:
# Create a SQLAlchemy engine for easier data loading
# This engine helps pandas communicate with MySQL
username = mysql_config['user']
password = mysql_config['password']
host = mysql_config['host']
database = mysql_config['database']

# Build the connection string
connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"
engine = create_engine(connection_string)

# Select only the columns that match our table schema
columns_to_load = [
    'ride_id', 'rideable_type', 'started_at', 'ended_at',
    'start_station_name', 'start_station_id', 
    'end_station_name', 'end_station_id',
    'start_lat', 'start_lng', 'end_lat', 'end_lng',
    'member_casual', 'ride_length', 'day_of_week', 'month', 'hour'
]

# Create a copy with only the columns we need
df_to_load = df[columns_to_load].copy()

# Load data into MySQL
# We load in chunks of 10,000 rows at a time to avoid memory issues
try:
    df_to_load.to_sql(
        name='bike_trips',           # Table name
        con=engine,                  # Database connection
        if_exists='replace',         # Replace table if it exists
        index=False,                 # Don't include pandas index
        chunksize=10000,             # Load 10,000 rows at a time
        method='multi'               # Use multi-row inserts for speed
    )
    print("Data loaded successfully into MySQL!")
    
except Exception as e:
    print(f"Error loading data: {e}")

Data loaded successfully into MySQL!


---

## Step 4: Verify Data Was Loaded Correctly

I'll check that all records were loaded successfully and view a few sample records to confirm everything looks correct.

In [10]:
# Verify the data was loaded correctly
try:
    # Connect to the database
    connection = mysql.connector.connect(**mysql_config)
    cursor = connection.cursor()
    
    # Count total records in the table
    cursor.execute("SELECT COUNT(*) FROM bike_trips")
    count_result = cursor.fetchone()
    count = count_result[0]
    print(f"Total records in database: {count:,}")
    
    # Fetch and display first 5 records
    cursor.execute("SELECT * FROM bike_trips LIMIT 5")
    print("\nFirst 5 records (showing first 5 columns):")
    
    # Loop through results and display
    results = cursor.fetchall()
    for row in results:
        # Print first 5 columns only for readability
        first_five_columns = row[:5]
        print(first_five_columns)
    
    print("\n✓ Data verification complete!")
    
    # Close cursor and connection
    cursor.close()
    connection.close()
    
except Exception as e:
    print(f"Error verifying data: {e}")

Total records in database: 4,859,019

First 5 records (showing first 5 columns):
('C1D650626C8C899A', 'electric_bike', '2024-01-12 15:30:27', '2024-01-12 15:37:59', 'Wells St & Elm St')
('EECD38BDB25BFCB0', 'electric_bike', '2024-01-08 15:45:46', '2024-01-08 15:52:59', 'Wells St & Elm St')
('F4A9CE78061F17F7', 'electric_bike', '2024-01-27 12:27:19', '2024-01-27 12:35:19', 'Wells St & Elm St')
('C96080812CD285C5', 'classic_bike', '2024-01-07 11:21:24', '2024-01-07 11:30:03', 'Wells St & Elm St')
('0EA7CB313D4F456A', 'classic_bike', '2024-01-05 14:44:12', '2024-01-05 14:53:06', 'Wells St & Elm St')

✓ Data verification complete!


---

## Step 5: Execute SQL Queries

Now I'll run analytical queries to answer business questions using SQL. This demonstrates my SQL skills for interviews.

In [13]:
# Create a helper function to run queries and display results
def run_query(query, description):
    """
    Executes a SQL query and displays the results.
    
    Parameters:
    - query: The SQL query to execute
    - description: A description of what the query does
    
    Returns:
    - df_result: A pandas dataframe with the query results
    """
    # Print the description
    print(f"\n{description}")
    print("=" * 70)
    
    # Connect to the database
    connection = mysql.connector.connect(**mysql_config)
    
    # Execute the query and get results as a dataframe
    df_result = pd.read_sql(query, connection)
    
    # Close the connection
    connection.close()
    
    # Display the results
    print(df_result.to_string(index=False))
    
    # Return the dataframe
    return df_result

### Query 1: Total Rides by Member Type

I'll calculate how many rides each member type took and what percentage of total rides they represent.

In [14]:
# Query 1: Total rides by member type
query1 = """
SELECT 
    member_casual,
    COUNT(*) AS total_rides,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM bike_trips), 2) AS percentage
FROM bike_trips
GROUP BY member_casual
"""

result1 = run_query(query1, "Query 1: Total Rides by Member Type")


Query 1: Total Rides by Member Type
member_casual  total_rides  percentage
       member      3274659       67.39
       casual      1584360       32.61


### Query 2: Average Ride Duration by Member Type

I'll compare the average, minimum, and maximum ride duration for each member type.

In [15]:
# Query 2: Average ride duration
query2 = """
SELECT 
    member_casual,
    ROUND(AVG(ride_length), 2) AS avg_duration_minutes,
    ROUND(MIN(ride_length), 2) AS min_duration,
    ROUND(MAX(ride_length), 2) AS max_duration
FROM bike_trips
GROUP BY member_casual
"""

result2 = run_query(query2, "Query 2: Average Ride Duration by Member Type")


Query 2: Average Ride Duration by Member Type
member_casual  avg_duration_minutes  min_duration  max_duration
       member                  9.23           1.0          24.0
       casual                 10.61           1.0          24.0


### Query 3: Weekday vs Weekend Usage

I'll analyze how usage patterns differ between weekdays and weekends for each member type.

In [16]:
# Query 3: Weekday vs Weekend
query3 = """
SELECT 
    member_casual,
    CASE 
        WHEN day_of_week IN (5, 6) THEN 'Weekend'
        ELSE 'Weekday'
    END AS day_type,
    COUNT(*) AS total_rides,
    ROUND(AVG(ride_length), 2) AS avg_duration
FROM bike_trips
GROUP BY member_casual, day_type
ORDER BY member_casual, day_type
"""

result3 = run_query(query3, "Query 3: Weekday vs Weekend Usage")


Query 3: Weekday vs Weekend Usage
member_casual day_type  total_rides  avg_duration
       casual  Weekday      1030945         10.22
       casual  Weekend       553415         11.32
       member  Weekday      2511698          9.11
       member  Weekend       762961          9.63


---

## Summary

I have successfully:
- ✓ Connected to MySQL server
- ✓ Created a new database called `cyclistic_db`
- ✓ Created a table called `bike_trips` with proper schema
- ✓ Loaded 4,859,019 records into the database
- ✓ Verified the data loaded correctly
- ✓ Executed analytical SQL queries to answer business questions

### Key Findings from SQL Queries:

1. **Member Distribution:**
   - Annual Members: 67.39% of rides
   - Casual Riders: 32.61% of rides

2. **Ride Duration:**
   - Casual riders take longer rides (10.61 min vs 9.23 min)
   - Both groups have rides ranging from 1 to 24 minutes

3. **Weekday vs Weekend:**
   - Members ride more on weekdays (commute pattern)
   - Casual riders have higher weekend usage (leisure pattern)
   - Weekend rides are longer on average for both groups
